# Running a 5-Point Calibration Scan Without OSO Using BITE

This notebook adapts the [Five Point Calibration Scan Controls](https://gitlab.com/ska-telescope/ska-jupyter-scripting/-/blob/main/notebooks/observing/MID_five_point_calibration_scan_controls.ipynb?ref_type=heads) notebook to work with PSI. Running this notebook requires a PSI namespace to be spun up with the CI argument `TMC_ENABLED` set to `true`.

## 1 Setup

## 1.1 Environment Setup

Start by importing all the libraries we'll be using for this notebook.

In [1]:
import json
import os
import random
import sys
import time
from datetime import date
from time import sleep

from tango import DevFailed, DeviceProxy

Until notebook clean up is merged need to use this for waiting for state

In [2]:
spinner = ["⣾", "⣽", "⣻", "⢿", "⡿", "⣟", "⣯", "⣷"]


def wait_for_state(device: DeviceProxy, desired_state, break_on_error=True) -> None:
    """Poll a tango device until either the given observation state is reached, or it throws an error.
    Arguments:
    device -- Tango Device to check
    desired_state -- The state which to break upon getting (number or state)
    break_on_error -- If set to False, will keeping running when getting an error status.
    """
    spinL = 0
    poll = 1
    while device.obsState != desired_state:
        if spinL < len(spinner) - 1:
            spinL += 1
        else:
            spinL = 0
        sleep(0.5)
        print(
            "\r",
            f"{spinner[spinL]} Poll# {poll}: Current state is {device.obsState.name}, waiting for {desired_state}...",
            end="",
        )
        if device.obsState == 9 and break_on_error:
            break
        poll += 1
    print(f"\nFinished with: {device.obsState.name}")

## 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [3]:
!kubectl get ns | grep ska-mid-psi

ci-ska-mid-psi-1357947816-alexschell                    Active   7m52s
ci-ska-mid-psi-1357947816-alexschell-sdp                Active   8m33s


And load it into the variables for the notebook:

In [4]:
namespace = "ci-ska-mid-psi-1357947816-alexschell"  # set to desired NS
simulation_mode = 0  # set to 1 to run in sim mode
target_boards_list = [2]  # assign boards
delay_model_filename = "delay_model.json"  # delay model file to use
test_id = "talon2 basic gaussian noise"  # Test to send config from

Next, we load all the other vars the notebook will use. These should not need to be changed for this run.

In [5]:
# Fully Qualified Domain Names for the devices to set up proxies
CSP_CONTROLLER_FQDN = "mid-csp/control/0"
CSP_SUBARRAY_FQDN = "mid-csp/subarray/01"

CENTRAL_NODE_FQDN = "ska_mid/tm_central/central_node"
LEAF_NODE_FQDN = "ska_mid/tm_leaf_node/csp_master"
TMC_SUBARRAY_FQDN = "ska_mid/tm_subarray_node/1"

CBF_CONTROLLER_FQDN = "mid_csp_cbf/sub_elt/controller"
CBF_SUBARRAY_FQDN = "mid_csp_cbf/sub_elt/subarray_01"

BITE_FQDN = "mid_csp_cbf/ec/bite"
DEPLOYER_FQDN = "mid_csp_cbf/ec/deployer"

# Tango host environment variable
TANGO_HOST = "databaseds-tango-base." + namespace + ".svc.cluster.local:10000"

# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIG = os.path.join(DATA_DIR, "mid_telescope/cbf")
CSP_CONFIG = os.path.join(DATA_DIR, "mid_telescope/csp")
TMC_CONFIG = os.path.join(DATA_DIR, "mid_telescope/tmc")
HW_CONFIG = os.path.join(COMMON_CONFIG, "hw_config")
SLIM_CONFIG = os.path.join(COMMON_CONFIG, "slim_config")
CBF_INPUT_DIR = os.path.join(COMMON_CONFIG, "cbf_input_data")
# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]
TARGET_BOARDS_STR = ",".join(str(x) for x in target_boards_list)
receptor_ids = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list))

Next, set the environment arg for TANGO HOST:

In [6]:
print("Will be using HOST: ", TANGO_HOST)
os.environ["TANGO_HOST"] = TANGO_HOST

Will be using HOST:  databaseds-tango-base.ci-ska-mid-psi-1357947816-alexschell.svc.cluster.local:10000


With all the file paths defined we can load in the JSON files and ensure they exist.

In [7]:
print("Getting files...")

INIT_SYS_PARAM_FILE = os.path.join(
    COMMON_CONFIG, "sys_params/initial_system_param.json"
)
ASSIGN_RESOURCES_FILE = os.path.join(TMC_CONFIG, "assign_resources.json")
ASSIGN_CSP_RESOURCES_FILE = os.path.join(CSP_CONFIG, "assign_resources.json")
CONFIGURE_SCAN_FILE = os.path.join(TMC_CONFIG, "configure_scan.json")
SCAN_FILE = os.path.join(TMC_CONFIG, "scan.json")
CSP_DELAY_MODEL_FILE = os.path.join(TMC_CONFIG, delay_model_filename)

CBF_INPUT_FILE = f"{CBF_INPUT_DIR}/cbf_input_data.json"
BITE_CONFIG_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/bite_configs.json"
FILTERS_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/filters.json"

DISH_CONFIG_FILE = f"{COMMON_CONFIG}/sys_params/load_dish_config.json"

START_CHANNEL = 0
END_CHANNEL = 14860
START_PORT = 21000

SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    SCAN_FILE,
    CSP_DELAY_MODEL_FILE,
    CBF_INPUT_FILE,
    BITE_CONFIG_FILE,
    FILTERS_FILE,
    DISH_CONFIG_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

print("Done")

Getting files...
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/sys_params/initial_system_param.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/assign_resources.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/csp/assign_resources.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/csp/assign_resources.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/configure_scan.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/scan.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/delay_model.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/cbf_input_data/cbf_input_data.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/cbf_input_data/bite_config_parameters/bite_c

Next, set up the hw config to match the boards in use:

In [8]:
if any(i > 4 for i in target_boards_list):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_boards_list = list(map(lambda x: x - 4, target_boards_list))

else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")
TARGET_BOARDS_STR = ",".join(str(x) for x in target_boards_list)

Using standard HW config
HW config: ✔️


If the HW config file exists, we can load it into the pod:

In [9]:
!kubectl cp $HW_CONFIG_FILE $namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

Next we can run the deployer to get everything needed for this run

### 1.3 Create Device Proxies

Using the FQDNs we set earlier, and with the pod spun up, we can create device proxies to the devices we'll be using and check the connection to them.

In [10]:
csp_controller = DeviceProxy(CSP_CONTROLLER_FQDN)
print(f"CSP Controller: {csp_controller.Status()}")
csp_subarray = DeviceProxy(CSP_SUBARRAY_FQDN)
print(f"CSP Subarray: {csp_subarray.Status()}")

tmc_central_node = DeviceProxy(CENTRAL_NODE_FQDN)
print(f"Central Node: {tmc_central_node.Status()}")
leaf_node = DeviceProxy(LEAF_NODE_FQDN)
print(f"Leaf Node: {leaf_node.Status()}")
tmc_subarray = DeviceProxy(TMC_SUBARRAY_FQDN)
print(f"TMC subarray Node: {tmc_subarray.Status()}")

bite = DeviceProxy(BITE_FQDN)
deployer = DeviceProxy(DEPLOYER_FQDN)

cbf_controller = DeviceProxy(CBF_CONTROLLER_FQDN)
print(f"CBF controller: {cbf_controller.Status()}")
cbf_subarray = DeviceProxy(CBF_SUBARRAY_FQDN)
print(f"CBF subarray: {cbf_subarray.Status()}")

CSP Controller: The device is in DISABLE state.
CSP Subarray: The device is in DISABLE state.
Central Node: The device is in ON state.
Leaf Node: The device is in ON state.
TMC subarray Node: The device is in ON state.
CBF controller: The device is in DISABLE state.
CBF subarray: The device is in DISABLE state.


### 1.4 Downloading Requirements via the Deployer

First, set the board we'll be deploying to and turn on the deployer device.

In [11]:
deployer.On()
deployer.targetTalons = target_boards_list
print(deployer.targetTalons)
deployer.generate_config_jsons()

[2]


Once started and configured, the required devices can then be downloaded.

In [12]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(3000)

Now configure the device database with the downloaded devices:

In [13]:
deployer.configure_db()

## 2 Setting up the Devices

### 2.1 Start up CSP Controller

Now setup the CSP proxy, and load in the relevant values:

In [28]:
# Set relevant values on the mid-csp controller
csp_controller.adminMode = 0
csp_subarray.adminMode = 0
sleep(1)
csp_controller.cbfSimulationMode = simulation_mode
csp_subarray.SimulationMode = simulation_mode
sleep(1)
if (
    csp_controller.read_attribute("adminMode").value == 0
    and csp_controller.read_attribute("cbfSimulationMode").value == 0
):
    print("Set to simulation mode off.")
elif (
    csp_controller.read_attribute("adminMode").value == 0
    and csp_controller.read_attribute("cbfSimulationMode").value == 1
):
    print("Set to simulation mode on.")
else:
    print("Error, couldn't set values!")

Set to simulation mode on.


Next we can load in the dish config file to the central node:

In [15]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = f.read()
tmc_central_node.LoadDishCfg(dish_config_json)

[array([2], dtype=int32), ['1719953381.9908297_104212624019405_LoadDishCfg']]

Now, load in the initial parameters file:

In [16]:
with open(INIT_SYS_PARAM_FILE, encoding="utf-8") as init_file:
    data = json.load(init_file)
print("Initial system parameter file:")
print(json.dumps(data, indent=1))

Initial system parameter file:
{
 "interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0",
 "dish_parameters": {
  "SKA001": {
   "vcc": 1,
   "k": 1108
  },
  "SKA036": {
   "vcc": 2,
   "k": 1108
  },
  "SKA063": {
   "vcc": 3,
   "k": 1100
  },
  "SKA100": {
   "vcc": 4,
   "k": 1101
  }
 }
}


In [17]:
# After confirming the file is correct load into the CBF
upload_result = cbf_controller.InitSysParam(json.dumps(data))

In [18]:
print(csp_controller.status())

The device is in OFF state.


With the required files loaded, the command to spin up the actual controller can be run:

In [19]:
controller_target = ["mid_csp_cbf/sub_elt/controller"]
csp_controller.set_timeout_millis(100000)
csp_controller.On(controller_target)
time.sleep(100)
print("CSP Controller State: ", csp_controller.State())
# csp_controller.set_timeout_millis(3000)

CSP Controller State:  ON


### 2.2 Assign Resources to CSP

In [20]:
with open(ASSIGN_CSP_RESOURCES_FILE, encoding="utf-8") as init_file:
    config_dict = json.load(init_file)
config_dict["dish"]["receptor_ids"] = receptor_ids
print(json.dumps(config_dict, indent=1))

{
 "interface": "https://schema.skao.int/ska-csp-assignresources/2.2",
 "subarray_id": 1,
 "dish": {
  "receptor_ids": [
   "SKA036"
  ]
 }
}


In [21]:
csp_subarray.AssignResources(json.dumps(config_dict))
wait_for_state(csp_subarray, 2)

 ⣽ Poll# 1: Current state is IDLE, waiting for 2...
Finished with: IDLE


### 2.3 Turning on the Telescope

TODO: Check if VCC mapping required here...

We can now check if the VCC mapping was loaded

In [ ]:
print(leaf_node.sourceDishVccConfig)

Now, turn the telescope itself on:

In [ ]:
tmc_central_node.TelescopeOn()

print(tmc_central_node.status())
print(csp_controller.status())

In [22]:
print("Verify device states:")
print(f"\tTMC central node: {tmc_central_node.State()}")
print(f"\tTMC subarray node: {tmc_subarray.State()}")
print(f"\tCSP controller node: {csp_controller.State()}")
print(f"\tCSP subarray node: {csp_subarray.State()}")
print(f"\tCBF subarray node: {cbf_subarray.State()}")

Verify device states:
	TMC central node: ON
	TMC subarray node: ON
	CSP controller node: ON
	CSP subarray node: ON
	CBF subarray node: ON


## 3 Controlling the 5-Point Scan from TMC

### 3.1 Set up JSON Files

TODO: Grab these from files, replace beams and anything else.

In [23]:
today = date.today().strftime("%Y%m%d")
random_id = random.randint(0, 99999)

beams = [
    {"beam_id": "vis0", "function": "visibilities"},
    {"beam_id": "pss1", "search_beam_id": 1, "function": "pulsar search"},
    {"beam_id": "pss2", "search_beam_id": 2, "function": "pulsar search"},
    {"beam_id": "pst1", "timing_beam_id": 1, "function": "pulsar timing"},
    {"beam_id": "pst2", "timing_beam_id": 2, "function": "pulsar timing"},
    {"beam_id": "vlbi1", "vlbi_beam_id": 1, "function": "vlbi"},
]

EXECUTION_BLOCK_ID = f"eb-testtmc-{today}-{random_id:05d}"
PROCESSING_BLOCK_ID_REALTIME = f"pb-testrealtime-{today}-{random_id:05d}"

long_basic_assign_json = {
    "interface": "https://schema.skao.int/ska-tmc-assignresources/2.1",
    "transaction_id": "txn-....-00001",
    "subarray_id": 1,
    "dish": {"receptor_ids": receptor_ids},
    "sdp": {
        "interface": "https://schema.skao.int/ska-sdp-assignres/0.4",
        "execution_block": {
            "eb_id": EXECUTION_BLOCK_ID,
            "max_length": 100.0,
            "context": {},
            "beams": beams,
            "scan_types": [
                {
                    "scan_type_id": ".default",
                    "beams": {
                        "vis0": {
                            "channels_id": "vis_channels",
                            "polarisations_id": "all",
                        },
                        "pss1": {
                            "field_id": "pss_field_0",
                            "channels_id": "pulsar_channels",
                            "polarisations_id": "all",
                        },
                        "pss2": {
                            "field_id": "pss_field_1",
                            "channels_id": "pulsar_channels",
                            "polarisations_id": "all",
                        },
                        "pst1": {
                            "field_id": "pst_field_0",
                            "channels_id": "pulsar_channels",
                            "polarisations_id": "all",
                        },
                        "pst2": {
                            "field_id": "pst_field_1",
                            "channels_id": "pulsar_channels",
                            "polarisations_id": "all",
                        },
                        "vlbi": {
                            "field_id": "vlbi_field",
                            "channels_id": "vlbi_channels",
                            "polarisations_id": "all",
                        },
                    },
                },
                {
                    "scan_type_id": "science",
                    "derive_from": ".default",
                    "beams": {"vis0": {"field_id": "field_a"}},
                },
            ],
            "channels": [
                {
                    "channels_id": "vis_channels",
                    "spectral_windows": [
                        {
                            "spectral_window_id": "fsp_1_channels",
                            "count": 744,
                            "start": 0,
                            "stride": 2,
                            "freq_min": 350000000.0,
                            "freq_max": 368000000.0,
                            "link_map": [[0, 0], [200, 1], [744, 2], [944, 3]],
                        },
                        {
                            "spectral_window_id": "fsp_2_channels",
                            "count": 744,
                            "start": 2000,
                            "stride": 1,
                            "freq_min": 360000000.0,
                            "freq_max": 368000000.0,
                            "link_map": [[2000, 4], [2200, 5]],
                        },
                        {
                            "spectral_window_id": "zoom_window_1",
                            "count": 744,
                            "start": 4000,
                            "stride": 1,
                            "freq_min": 360000000.0,
                            "freq_max": 361000000.0,
                            "link_map": [[4000, 6], [4200, 7]],
                        },
                    ],
                },
                {
                    "channels_id": "pulsar_channels",
                    "spectral_windows": [
                        {
                            "spectral_window_id": "pulsar_fsp_channels",
                            "count": 744,
                            "start": 0,
                            "freq_min": 350000000.0,
                            "freq_max": 368000000.0,
                        }
                    ],
                },
            ],
            "polarisations": [
                {"polarisations_id": "all", "corr_type": ["XX", "XY", "YY", "YX"]}
            ],
            "fields": [
                {
                    "field_id": "field_a",
                    "phase_dir": {
                        "ra": [123, 0.1],
                        "dec": [80, 0.1],
                        "reference_time": "...",
                        "reference_frame": "ICRF3",
                    },
                    "pointing_fqdn": "low-tmc/telstate/0/pointing",
                }
            ],
        },
        "processing_blocks": [
            {
                "pb_id": PROCESSING_BLOCK_ID_REALTIME,
                "sbi_ids": ["sbi-mvp01-20200325-00001"],
                "script": {
                    "kind": "realtime",
                    "name": "test-receive-addresses",  # "vis-receive",
                    "version": "0.7.1",  # "2.2.0"
                },
                "parameters": {},
            }
        ],
        "resources": {
            "csp_links": [1, 2, 3, 4],
            "receptors": ["SKA001", "SKA036", "SKA063", "SKA100"],
            "receive_nodes": 10,
        },
    },
}

In [24]:
long_basic_configure_json = {
    "interface": "https://schema.skao.int/ska-tmc-configure/2.2",
    "transaction_id": "txn-....-00002",
    "pointing": {
        "target": {
            "reference_frame": "ICRS",
            "target_name": "Polaris Australis",
            "ra": "21:08:47.92",
            "dec": "-88:57:22.9",  # ,
            # "ca_offset_arcsec": 0.0,
            # "ie_offset_arcsec": 0.0
        }
    },
    "dish": {"receiver_band": "1"},
    "csp": {
        "interface": "https://schema.skao.int/ska-csp-configure/2.0",
        "subarray": {"subarray_name": "science period 23"},
        "common": {
            "config_id": "sbi-mvp01-20200325-00001-science",
            "frequency_band": "1",
            "subarray_id": 1,
        },
        "cbf": {
            "fsp": [
                {
                    "fsp_id": 1,
                    "function_mode": "CORR",
                    "frequency_slice_id": 1,
                    "integration_factor": 1,
                    "zoom_factor": 0,
                    "channel_averaging_map": [[0, 2], [744, 0]],
                    "channel_offset": 0,
                    "output_link_map": [[0, 0], [200, 1]],
                },
                {
                    "fsp_id": 2,
                    "function_mode": "CORR",
                    "frequency_slice_id": 2,
                    "integration_factor": 1,
                    "zoom_factor": 1,
                    "channel_averaging_map": [[0, 2], [744, 0]],
                    "channel_offset": 744,
                    "output_link_map": [[0, 4], [200, 5]],
                    "zoom_window_tuning": 650000,
                },
            ],
            "vlbi": {},
        },
        "pss": {},
        "pst": {},
    },
    "sdp": {
        "interface": "https://schema.skao.int/ska-sdp-configure/0.4",
        "scan_type": "science",
    },
    "tmc": {
        "scan_duration": 10.0,
        # "partial_configuration": False
    },
}

### 3.2 Assign Resources to the Telescope and Configure Subarray

In [25]:
tmc_subarray.AssignResources(json.dumps(long_basic_assign_json))
wait_for_state(tmc_subarray, 2)

 ⣟ Poll# 5: Current state is IDLE, waiting for 2...r 2...
Finished with: IDLE


In [26]:
tmc_subarray.Configure(json.dumps(long_basic_configure_json))
wait_for_state(tmc_subarray, 2)

 ⣽ Poll# 1: Current state is IDLE, waiting for 2...
Finished with: IDLE


### 3.3 Setup BITE Data

TODO: Investigate LSTV_gen going to unkown state during setup

For this version of the notebook, we look to BITE to generate the data we'll use for scanning.

First, load in all the data we'll use

In [ ]:
with open(CBF_INPUT_FILE, encoding="utf-8") as f:
    cbf_input_json = json.load(f)["cbf_input_data"][test_id]
    cbf_input_data = json.dumps(cbf_input_json)
    print("CBF Input Data used to generate BITE data:\n")
    print(cbf_input_data)

bite.load_cbf_input_data(cbf_input_data)

In [ ]:
with open(BITE_CONFIG_FILE, encoding="utf-8") as f:
    bite_config_data = json.dumps(json.load(f))
    print("BITE configs:\n")
    print(bite_config_data)

bite.load_bite_config_data(bite_config_data)

In [ ]:
with open(FILTERS_FILE, encoding="utf-8") as f:
    filter_data = json.dumps(json.load(f))
    print("Filters:\n")
    print(filter_data)

bite.load_filter_data(filter_data)

With these loaded, we can actually generate the BITE data:

In [ ]:
# kubectl logs -n $NS ds-bite-bite-0 -f

bite.set_timeout_millis(240000)
bite.generate_bite_data()
bite.set_timeout_millis(3000)

## 4 Running the 5-Point Scan

Now we can actually run the scans:

### 4.1 Run Scan 1: No Offset

In [ ]:
long_basic_scan_json = {
    "interface": "https://schema.skao.int/ska-tmc-scan/2.1",
    "transaction_id": "txn-....-00003",
    "scan_id": 1,
}

In [ ]:
tmc_subarray.Scan(json.dumps(long_basic_scan_json))

In [ ]:
wait_for_state(tmc_subarray, 5)
wait_for_state(tmc_subarray, 4)

Once this passes, we can then run the scans for offsets

### 4.2 Run Offset Scans 2-5

Now, loop through the offset scans, using the combos to set the scan json

In [ ]:
count = 4
for offset in SCAN_COMBOS:
    print(f"setting offset to: {offset}")
    # configure the JSON to use this offset
    partial_configure_json = {
        "interface": "https://schema.skao.int/ska-tmc-configure/2.2",
        "transaction_id": f"txn-....-0000{count}",
        "pointing": {
            "target": {"ca_offset_arcsec": offset[0], "ie_offset_arcsec": offset[1]}
        },
        "tmc": {"partial_configuration": True},
    }
    print(json.dumps(partial_configure_json, indent=1))
    tmc_subarray.Scan(json.dumps(partial_configure_json))
    wait_for_state(tmc_subarray, 5)
    wait_for_state(tmc_subarray, 4)

    count += 1
    print("============================")
print("done offsets)")

## 5 Cleanup:

In [ ]:
tmc_subarray.GoToIdle()
tmc_subarray.ReleaseAllResources()